In [1]:
import itertools as it

from matplotlib import pyplot as plt
from nbmetalog import nbmetalog as nbm
import numpy as np
import pandas as pd
import seaborn as sns
from teeplot import teeplot as tp


In [2]:
# prints metadata about notebook runtime
nbm.print_metadata()


context: ci
hostname: fv-az1108-768
interpreter: 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
nbcellexec: null
nbname: postprocess-bias
nbpath: /home/runner/work/hstrat-evolutionary-inference/hstrat-evolutionary-inference/binder/simple-splayed/postprocess-bias.ipynb
revision: null
session: dd0dff52-2ca7-413e-9846-71f8adeaed58
timestamp: 2024-06-16T00:36:21Z00:00


IPython==8.13.0
keyname==0.5.2
yaml==6.0
matplotlib==3.7.1
nbmetalog==0.2.6
numpy==1.21.6
pandas==1.3.5
seaborn==0.12.1
teeplot==1.0.1
re==2.2.1
comm==0.2.2
logging==0.5.1.2
psutil==5.9.8
zmq==24.0.1
json==2.0.9
ipykernel==6.29.3


In [3]:
df = pd.read_csv("https://osf.io/xd95a/download")
dfdigest = np.bitwise_xor.reduce(
    pd.util.hash_pandas_object(df),
)
print("{:x}".format(dfdigest))
df


d5fa12f6738c655


,a,epoch,mut_distn,num_generations,num_islands,num_niches,p_island_migration,p_niche_invasion,population_size,replicate,...,variance_pairwise_distance,variance_pairwise_distance_branch_only,mrca_depth,sackin_index,mean_evolutionary_distinctiveness,sum_evolutionary_distinctiveness,variance_evolutionary_distinctiveness,resolution,subsampling-fraction,trie-postprocess
0,collapsed-phylogeny,7,np.random.standard_normal,32768,1024,4,0.01,3.051758e-06,32768,20,...,31.886538,31.886538,1,609638,78.882446,2.584820e+06,1348.379396,NaN,NaN,NaN
1,collapsed-phylogeny,7,np.random.standard_normal,32768,1024,4,0.01,3.051758e-06,32768,4,...,28.357566,28.357566,1,588616,81.707306,2.677385e+06,1030.406615,NaN,NaN,NaN
2,collapsed-phylogeny,7,np.random.standard_normal,32768,1024,4,0.01,3.051758e-06,32768,41,...,28.516943,28.516943,0,592371,83.217377,2.726867e+06,1170.361609,NaN,NaN,NaN
3,collapsed-phylogeny,7,np.random.standard_normal,32768,1024,4,0.01,3.051758e-06,32768,47,...,30.233129,30.233129,0,586736,86.538544,2.835695e+06,1284.142442,NaN,NaN,NaN
4,collapsed-phylogeny,7,np.random.standard_normal,32768,1024,4,0.01,3.051758e-06,32768,15,...,29.675131,29.675131,1,600764,76.277557,2.499463e+06,1545.456043,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104995,reconstructed-tree,2,np.random.standard_normal,32768,1,4,0.01,3.051758e-08,32768,10,...,28.560595,28.560595,1,3547,99.163609,3.242650e+04,282.320163,30.0,0.01,naive
104996,reconstructed-tree,2,np.random.standard_normal,32768,1,4,0.01,3.051758e-08,32768,16,...,20.627240,20.627240,0,3331,1893.146789,6.190590e+05,250803.885040,100.0,0.01,rollback
104997,reconstructed-tree,2,np.random.standard_normal,32768,1,4,0.01,3.051758e-08,32768,1,...,114.169456,114.169456,1,821866,5.744125,1.882235e+05,6.163179,100.0,1.00,rollback
104998,reconstructed-tree,2,np.random.standard_normal,32768,1,4,0.01,3.051758e-08,32768,10,...,36.528638,36.528638,0,623208,28.659882,9.391270e+05,116.954707,3.0,1.00,naive


In [4]:
# filter out incomplete sets of replicates
df1 = df[
    df["a"] == "reconstructed-tree"
].groupby(
    [
        'epoch',
        'mut_distn',
        'num_islands',
        'num_niches',
        'p_island_migration',
        'p_niche_invasion',
        'population_size',
        'tournament_size',
        'treatment',
        'a',
        'resolution',
        'subsampling-fraction',
        'trie-postprocess',
    ],
    dropna=False,
).filter(
    lambda x: len(x) == 50,
).reset_index(drop=True)

df1["a"].unique()


array(['reconstructed-tree'], dtype=object)

In [5]:
# filter out incomplete sets of replicates
df2 = df[
    df["a"] == "collapsed-phylogeny"
].groupby(
    [
        'epoch',
        'mut_distn',
        'num_islands',
        'num_niches',
        'p_island_migration',
        'p_niche_invasion',
        'population_size',
        'tournament_size',
        'treatment',
        'a',
    ],
    dropna=False,
).filter(
    lambda x: len(x) == 50,
).reset_index(drop=True)

df2["a"].unique()


array(['collapsed-phylogeny'], dtype=object)

In [6]:
df = pd.concat([df1, df2], ignore_index=True)
df


,a,epoch,mut_distn,num_generations,num_islands,num_niches,p_island_migration,p_niche_invasion,population_size,replicate,...,variance_pairwise_distance,variance_pairwise_distance_branch_only,mrca_depth,sackin_index,mean_evolutionary_distinctiveness,sum_evolutionary_distinctiveness,variance_evolutionary_distinctiveness,resolution,subsampling-fraction,trie-postprocess
0,reconstructed-tree,7,np.random.exponential,32768,1024,4,0.01,3.051758e-08,32768,46,...,30.592668,30.592668,0,601771,118.941956,3.897490e+06,4.181015e+03,10.0,1.00,rollback
1,reconstructed-tree,7,np.random.exponential,32768,1024,4,0.01,3.051758e-08,32768,30,...,25.488540,25.488540,0,576666,123.441128,4.044919e+06,5.416807e+03,30.0,1.00,expected
2,reconstructed-tree,7,np.random.exponential,32768,1024,4,0.01,3.051758e-08,32768,32,...,16.790542,16.790542,0,3177,8880.562691,2.903944e+06,2.375216e+07,3.0,0.01,rollback
3,reconstructed-tree,7,np.random.exponential,32768,1024,4,0.01,3.051758e-08,32768,1,...,25.969854,25.969854,0,579202,122.678925,4.019943e+06,3.007903e+03,10.0,1.00,naive
4,reconstructed-tree,7,np.random.exponential,32768,1024,4,0.01,3.051758e-08,32768,17,...,20.596500,20.596500,0,3242,10902.110092,3.564990e+06,3.698008e+07,10.0,0.01,naive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104995,collapsed-phylogeny,2,np.random.exponential,32768,1024,4,0.01,3.051758e-06,32768,35,...,31.222858,31.222858,0,597571,62.567078,2.050198e+06,6.991898e+02,NaN,NaN,NaN
104996,collapsed-phylogeny,2,np.random.exponential,32768,1024,4,0.01,3.051758e-06,32768,30,...,33.379170,33.379170,0,593800,54.291748,1.779032e+06,5.015938e+02,NaN,NaN,NaN
104997,collapsed-phylogeny,2,np.random.exponential,32768,1024,4,0.01,3.051758e-06,32768,49,...,36.683984,36.683984,0,600262,61.928589,2.029276e+06,1.052942e+03,NaN,NaN,NaN
104998,collapsed-phylogeny,2,np.random.exponential,32768,1024,4,0.01,3.051758e-06,32768,42,...,27.494275,27.494275,0,580657,65.404114,2.143162e+06,8.707620e+02,NaN,NaN,NaN


In [7]:
# phylometrics whittled down to descriptive, non-redundant set
df["colless-like index"] = df["colless_like_index"]
df["mean pairwise distance"] = df["mean_pairwise_distance"]
df["sum pairwise distance"] = df["sum_distance"]
df["mean evolutionary distinctiveness"] = df[
    "mean_evolutionary_distinctiveness"
]

phylometrics = [
    #     'diversity',
    "colless-like index",
    #     'average_depth',
    #     'average_origin_time',
    #     'average_origin_time_normalized',
    #     'max_depth',
    "mean pairwise distance",
    #     'mean_pairwise_distance_branch_only',
    #     'num_active',
    "sum pairwise distance",
    # "num_ancestors",
    #     'num_outside',
    #     'num_roots',
    #     'num_taxa',
    #     'phylogenetic_diversity',
    #     'sum_pairwise_distance',
    #     'sum_pairwise_distance_branch_only',
    #     'total_orgs',
    #     'tree_size',
    #     'variance_pairwise_distance',
    #     'variance_pairwise_distance_branch_only',
    #     'mrca_depth',
    #     'sackin_index',
    "mean evolutionary distinctiveness",
    #     'sum_evolutionary_distinctiveness',
    #     'variance_evolutionary_distinctiveness',
]


In [8]:
methodological_variables = [
    "resolution",
    "trie-postprocess",
]


In [9]:
evolutionary_variables = [
    "num_islands",
    "num_niches",
    "tournament_size",
]


In [10]:
sensitivity_analysis_variables = [
    "epoch",
    "mut_distn",
]


In [11]:
fixed_variables = [
    "num_generations",  # homogeneous (num generations per epoch)
    "p_island_migration",  # homogeneous
    "population_size",  # homogeneous
    "p_niche_invasion",  # only use 3.0517578125e-08
    "subsampling-fraction",  # only use 1.0
]


In [12]:
# ensure fixed variables
df = df[
    (df["p_island_migration"] == 0.01)
    & (df["p_niche_invasion"] == 3.0517578125e-08)
    & ((df["subsampling-fraction"] == 1.0) | df["subsampling-fraction"].isna())
].copy()

for fixed_variable in fixed_variables:
    assert len(df[fixed_variable].dropna().unique()) == 1, excluded_variable

df


,a,epoch,mut_distn,num_generations,num_islands,num_niches,p_island_migration,p_niche_invasion,population_size,replicate,...,mean_evolutionary_distinctiveness,sum_evolutionary_distinctiveness,variance_evolutionary_distinctiveness,resolution,subsampling-fraction,trie-postprocess,colless-like index,mean pairwise distance,sum pairwise distance,mean evolutionary distinctiveness
0,reconstructed-tree,7,np.random.exponential,32768,1024,4,0.01,3.051758e-08,32768,46,...,118.941956,3.897490e+06,4181.014677,10.0,1.0,rollback,305700.072677,32.806890,3.897490e+06,118.941956
1,reconstructed-tree,7,np.random.exponential,32768,1024,4,0.01,3.051758e-08,32768,30,...,123.441128,4.044919e+06,5416.806830,30.0,1.0,expected,262602.344292,32.469527,4.044919e+06,123.441128
3,reconstructed-tree,7,np.random.exponential,32768,1024,4,0.01,3.051758e-08,32768,1,...,122.678925,4.019943e+06,3007.903179,10.0,1.0,naive,272137.093188,32.216036,4.019943e+06,122.678925
5,reconstructed-tree,7,np.random.exponential,32768,1024,4,0.01,3.051758e-08,32768,45,...,124.094620,4.066332e+06,7149.002452,30.0,1.0,rollback,301925.210223,32.466066,4.066332e+06,124.094620
7,reconstructed-tree,7,np.random.exponential,32768,1024,4,0.01,3.051758e-08,32768,45,...,124.032479,4.064296e+06,7129.516815,100.0,1.0,expected,302162.494582,32.471554,4.064296e+06,124.032479
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104945,collapsed-phylogeny,0,np.random.standard_normal,32768,1024,4,0.01,3.051758e-08,32768,26,...,76.060944,2.492365e+06,1132.757139,NaN,NaN,NaN,581312.233566,34.051311,2.492365e+06,76.060944
104946,collapsed-phylogeny,0,np.random.standard_normal,32768,1024,4,0.01,3.051758e-08,32768,4,...,79.029419,2.589636e+06,2815.081067,NaN,NaN,NaN,602915.315959,34.849989,2.589636e+06,79.029419
104947,collapsed-phylogeny,0,np.random.standard_normal,32768,1024,4,0.01,3.051758e-08,32768,31,...,78.280975,2.565111e+06,1749.840662,NaN,NaN,NaN,573574.977471,34.613061,2.565111e+06,78.280975
104948,collapsed-phylogeny,0,np.random.standard_normal,32768,1024,4,0.01,3.051758e-08,32768,39,...,75.899933,2.487089e+06,1734.873002,NaN,NaN,NaN,512110.674312,33.510621,2.487089e+06,75.899933


In [13]:
# exclude extraneous variable values
df = df[(df["a"] != "consolidated-phylogeny")].copy()

df


,a,epoch,mut_distn,num_generations,num_islands,num_niches,p_island_migration,p_niche_invasion,population_size,replicate,...,mean_evolutionary_distinctiveness,sum_evolutionary_distinctiveness,variance_evolutionary_distinctiveness,resolution,subsampling-fraction,trie-postprocess,colless-like index,mean pairwise distance,sum pairwise distance,mean evolutionary distinctiveness
0,reconstructed-tree,7,np.random.exponential,32768,1024,4,0.01,3.051758e-08,32768,46,...,118.941956,3.897490e+06,4181.014677,10.0,1.0,rollback,305700.072677,32.806890,3.897490e+06,118.941956
1,reconstructed-tree,7,np.random.exponential,32768,1024,4,0.01,3.051758e-08,32768,30,...,123.441128,4.044919e+06,5416.806830,30.0,1.0,expected,262602.344292,32.469527,4.044919e+06,123.441128
3,reconstructed-tree,7,np.random.exponential,32768,1024,4,0.01,3.051758e-08,32768,1,...,122.678925,4.019943e+06,3007.903179,10.0,1.0,naive,272137.093188,32.216036,4.019943e+06,122.678925
5,reconstructed-tree,7,np.random.exponential,32768,1024,4,0.01,3.051758e-08,32768,45,...,124.094620,4.066332e+06,7149.002452,30.0,1.0,rollback,301925.210223,32.466066,4.066332e+06,124.094620
7,reconstructed-tree,7,np.random.exponential,32768,1024,4,0.01,3.051758e-08,32768,45,...,124.032479,4.064296e+06,7129.516815,100.0,1.0,expected,302162.494582,32.471554,4.064296e+06,124.032479
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104945,collapsed-phylogeny,0,np.random.standard_normal,32768,1024,4,0.01,3.051758e-08,32768,26,...,76.060944,2.492365e+06,1132.757139,NaN,NaN,NaN,581312.233566,34.051311,2.492365e+06,76.060944
104946,collapsed-phylogeny,0,np.random.standard_normal,32768,1024,4,0.01,3.051758e-08,32768,4,...,79.029419,2.589636e+06,2815.081067,NaN,NaN,NaN,602915.315959,34.849989,2.589636e+06,79.029419
104947,collapsed-phylogeny,0,np.random.standard_normal,32768,1024,4,0.01,3.051758e-08,32768,31,...,78.280975,2.565111e+06,1749.840662,NaN,NaN,NaN,573574.977471,34.613061,2.565111e+06,78.280975
104948,collapsed-phylogeny,0,np.random.standard_normal,32768,1024,4,0.01,3.051758e-08,32768,39,...,75.899933,2.487089e+06,1734.873002,NaN,NaN,NaN,512110.674312,33.510621,2.487089e+06,75.899933


In [14]:
df["num_niches"].unique()


array([4, 8, 1])

In [15]:
df["num_islands"].unique()


array([1024,    1])

In [16]:
df["tournament_size"].unique()


array([2, 1, 4])

In [17]:
df["a"].unique()


array(['reconstructed-tree', 'collapsed-phylogeny'], dtype=object)

In [18]:
df["regime"] = df.apply(
    lambda row: {
        (1, 1, 2): "plain",
        (1, 1, 1): "weak selection",
        (1, 1, 4): "strong selection",
        (1, 4, 2): "ecology",
        (1, 8, 2): "rich ecology",
        (1024, 1, 2): "spatial structure",
    }.get(
        tuple(row[["num_islands", "num_niches", "tournament_size"]]),
        np.nan,  # default
    ),
    axis="columns",
)
df = df.dropna(axis="index", subset=["regime"])
df["regime"].unique()


array(['rich ecology', 'spatial structure', 'weak selection',
       'strong selection', 'plain', 'ecology'], dtype=object)

In [19]:
df[df["a"] == "collapsed-phylogeny"]["regime"].unique()


array(['ecology', 'weak selection', 'plain', 'strong selection',
       'spatial structure', 'rich ecology'], dtype=object)

In [20]:
for setup, setup_df in df.groupby(sensitivity_analysis_variables):
    print(f"{setup=}")
    for regime, regime_df in setup_df.groupby(["regime"]):
        ground_truth = regime_df[(regime_df["a"] == "collapsed-phylogeny")][
            "average_origin_time_normalized"
        ].mean()
        print(f"   regime {ground_truth=}")

        for group, group_df in regime_df[
            (regime_df["a"] == "reconstructed-tree")
        ].groupby(["trie-postprocess", "resolution"]):
            err = (
                group_df["average_origin_time_normalized"].mean()
                - ground_truth
            )
            print("      ", group, len(group_df), err)


setup=(0, 'np.random.exponential')
   regime ground_truth=32765.118097476116
       ('expected', 3.0) 50 -0.025060253941774135
       ('expected', 10.0) 50 0.000995164049527375
       ('expected', 30.0) 50 -0.00018116798673872836
       ('expected', 100.0) 50 -0.0008352655859198421
       ('naive', 3.0) 50 -0.01729209265249665
       ('naive', 10.0) 50 0.007817926580173662
       ('naive', 30.0) 50 0.006332590721285669
       ('naive', 100.0) 50 0.005748161260271445
       ('rollback', 3.0) 50 -0.021631519517541165
       ('rollback', 10.0) 50 0.003895992922480218
       ('rollback', 30.0) 50 0.0024262215119961184
       ('rollback', 100.0) 50 0.0018417920473439153
   regime ground_truth=32766.119956053346
       ('expected', 3.0) 50 -0.00913880377629539
       ('expected', 10.0) 50 -0.0010293127015756909
       ('expected', 30.0) 50 -0.0007953567182994448
       ('expected', 100.0) 50 -0.0007405960095638875
       ('naive', 3.0) 50 -0.0020862453056906816
       ('naive', 10.0) 50 0.00